# 01 - Ingest PubMed Records

## Goal

Ingest PubMed records (dentistry focus): query by year using NCBI E-utilities, fetch MEDLINE XML into `data/raw/`. We'll be polite (rate limits), resumable, and explicit about scope.


## Why This Step Matters

**Reproducibility** is the foundation of scientific computing. By explicitly:

- Documenting our query terms
- Respecting NCBI rate limits (with `NCBI_EMAIL` and `NCBI_API_KEY`)
- Storing raw XML for provenance

...we ensure anyone can rebuild this dataset from scratch.

### How E-utilities Work

1. **ESearch** → returns list of PMIDs matching query
2. **EFetch** → retrieves full MEDLINE XML for those PMIDs

### Example XML Structure

```xml
<PubmedArticle>
  <MedlineCitation>
    <PMID>12345678</PMID>
    <Article>
      <ArticleTitle>Effect of dental implants...</ArticleTitle>
      <Abstract><AbstractText>This study...</AbstractText></Abstract>
    </Article>
    <MeshHeadingList>
      <MeshHeading><DescriptorName>Dental Implants</DescriptorName></MeshHeading>
    </MeshHeadingList>
  </MedlineCitation>
  <PubmedData>
    <PublicationTypeList>
      <PublicationType>Randomized Controlled Trial</PublicationType>
    </PublicationTypeList>
  </PubmedData>
</PubmedArticle>
```


In [69]:
# === TODO (you code this) ===
import os
import yaml
from pathlib import Path
from tqdm import tqdm
import sys
sys.path.append('..')
from src.utils import eutils_get
import os
from dotenv import load_dotenv


In [70]:
# === TODO (you code this) ===
# Goal: Read YAML config into a dict.
# Hints:
# 1) Use yaml.safe_load() on the file handle
# 2) Config should have: start_year, end_year, term_template, batch_ids, batch_fetch
# Acceptance:
# - config dict contains all 5 keys
# - start_year and end_year are integers

def load_config(path="../configs/query.yaml"):
    """Load YAML configuration."""
    # TODO
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
        print(config["start_year"])
        print(config["end_year"])
        print(config["term_template"])
        print(config["batch_ids"])
        print(config["batch_fetch"])
    return config

config = load_config()


2018
2025
(dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants"[MeSH Terms] OR "dental caries"[MeSH Terms] OR "periodontics"[MeSH Terms] OR "orthodontics"[MeSH Terms] OR "endodontics"[MeSH Terms] OR "prosthodontics"[MeSH Terms] OR "oral surgery"[MeSH Terms] OR "tooth"[MeSH Terms] OR "gingiva"[MeSH Terms] OR "periodontal diseases"[MeSH Terms] OR "oral hygiene"[MeSH Terms] OR "root canal therapy"[MeSH Terms] OR "dental implant"[Title/Abstract] OR "dental caries"[Title/Abstract] OR "tooth decay"[Title/Abstract] OR "periodontal disease"[Title/Abstract] OR "gingival"[Title/Abstract] OR "root canal"[Title/Abstract] OR "oral surgery"[Title/Abstract] OR "orthodontic"[Title/Abstract] OR "endodontic"[Title/Abstract] OR "prosthodontic"[Title/Abstract] OR "dental restoration"[Title/Abstract] OR "oral health"[Title/Abstract]) AND ({{year}}[PDAT]:{{year}}[PDAT]) NOT ("arthroplasty"[Title/Abstract] OR "knee prosthesis"[Title/Abstract] OR "hip prosthesis

In [71]:
# === TODO (you code this) ===
# Goal: Set NCBI credentials from environment variables.
# Hints:
# 1) Use os.getenv() with sensible defaults
# 2) Without API key: 3 req/sec; with key: 10 req/sec
# Acceptance:
# - NCBI_EMAIL and NCBI_API_KEY variables defined
# - Defaults provided if env vars missing

# Set NCBI_EMAIL and NCBI_API_KEY with defaults
load_dotenv('../.env')
NCBI_EMAIL = os.getenv('NCBI_EMAIL', 'your_email@example.com')
NCBI_API_KEY = os.getenv('NCBI_API_KEY', None)

if not os.getenv('NCBI_EMAIL'):
    print("Warning: NCBI_EMAIL not found in environment variables. Using default.")
else:
    print("✅ NCBI_EMAIL found in environment variables.")

if not os.getenv('NCBI_API_KEY'):
    print("Warning: NCBI_API_KEY not found in environment variables. API rate limited to 3 req/sec.")
else:
    print("✅ NCBI_API_KEY found in environment variables.")


✅ NCBI_EMAIL found in environment variables.
✅ NCBI_API_KEY found in environment variables.


In [72]:
# === TODO (you code this) ===
# Goal: Create data/raw directory if it doesn't exist.
# Hints:
# 1) Use Path().mkdir() with appropriate flags
# Acceptance:
# - Directory exists after running
# - No error if directory already exists

# TODO: Create ../data/raw directory
if not os.path.exists('../data/raw'):
    os.makedirs('../data/raw')
    print("✅ Created ../data/raw directory.")
else:
    print("✅ ../data/raw directory already exists.")



✅ ../data/raw directory already exists.


## Build the Yearly Query

We'll loop through years and construct queries like:

```
(dentistry[MeSH Terms] OR dental[Title/Abstract] OR ...) AND (2018[PDAT]:2018[PDAT])
```


In [73]:
# === TODO (you code this) ===
# Goal: Build the PubMed query string for a given year.
# Hints:
# 1) Replace {{year}} placeholder in term_template
# 2) Return complete query string ready for ESearch
# Acceptance:
# - Function build_query(year:int, template:str) -> str
# - Query for 2022 contains "2022[PDAT]:2022[PDAT]"

def build_query(year, template):
    """Build PubMed query string for a given year."""
    # Replace year placeholder
    query = template.replace('{{year}}', str(year))
    
    # Remove comment lines (lines starting with #) and clean whitespace
    lines = query.split('\n')
    cleaned_lines = [line for line in lines if not line.strip().startswith('#')]
    query = ' '.join(cleaned_lines)
    
    # Final cleanup: collapse multiple spaces to single space
    query = ' '.join(query.split())
    
    return query

# Test the function
test_query = build_query(2022, config['term_template'])
print(test_query)

# Verify it replaced both occurrences
assert '{{year}}' not in test_query  # Should pass - no more {{year}}
assert '2022' in test_query  # Should pass - year is in there
print("✅ build_query works correctly!")


(dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants"[MeSH Terms] OR "dental caries"[MeSH Terms] OR "periodontics"[MeSH Terms] OR "orthodontics"[MeSH Terms] OR "endodontics"[MeSH Terms] OR "prosthodontics"[MeSH Terms] OR "oral surgery"[MeSH Terms] OR "tooth"[MeSH Terms] OR "gingiva"[MeSH Terms] OR "periodontal diseases"[MeSH Terms] OR "oral hygiene"[MeSH Terms] OR "root canal therapy"[MeSH Terms] OR "dental implant"[Title/Abstract] OR "dental caries"[Title/Abstract] OR "tooth decay"[Title/Abstract] OR "periodontal disease"[Title/Abstract] OR "gingival"[Title/Abstract] OR "root canal"[Title/Abstract] OR "oral surgery"[Title/Abstract] OR "orthodontic"[Title/Abstract] OR "endodontic"[Title/Abstract] OR "prosthodontic"[Title/Abstract] OR "dental restoration"[Title/Abstract] OR "oral health"[Title/Abstract]) AND (2022[PDAT]:2022[PDAT]) NOT ("arthroplasty"[Title/Abstract] OR "knee prosthesis"[Title/Abstract] OR "hip prosthesis"[Title/Abstract] 

## ESearch: Get PMIDs

Use `esearch.fcgi` to get a list of PMIDs matching our query.


In [74]:
# === TODO (you code this) ===
# Goal: Call NCBI ESearch to get PMIDs for a query.
# Hints:
# 1) Use eutils_get() helper with 'esearch.fcgi' endpoint
# 2) Return JSON response (see NCBI E-utilities docs for params)
# 3) Include email and api_key in params for rate limit compliance
# Acceptance:
# - Function esearch(query, retmax, retstart) returns dict
# - Response contains 'esearchresult' with 'idlist'

def esearch(query, retmax=500, retstart=0):
    """Query NCBI ESearch and return JSON response."""
    params = {
        'db': 'pubmed',
        'term': query,
        'retmax': retmax,
        'retstart': retstart,
        'retmode': 'json',  # ⚠️ CRITICAL: Must specify JSON format
        'email': NCBI_EMAIL,
        'api_key': NCBI_API_KEY
    }
    
    # Debug: Print what we're sending
    print(f"🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi")
    print(f"📝 Query preview: {query[:100]}...")
    
    response = eutils_get('esearch.fcgi', params=params)
    
    # Debug: Check what we got back
    print(f"✅ Status code: {response.status_code}")
    print(f"📦 Content type: {response.headers.get('content-type')}")
    print(f"📄 First 200 chars of response:\n{response.text[:200]}")
    
    return response.json()

# Test the function
# Test with a simple query first
test_query = build_query(2022, config['term_template'])
print(f"Query length: {len(test_query)} characters")

try:
    result = esearch(test_query, retmax=10, retstart=0)  # Start with just 10 results
    print(f"✅ Success! Found {result['esearchresult']['count']} results")
except Exception as e:
    print(f"❌ Error: {e}")




Query length: 1115 characters
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...
✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"20465","retmax":"10","retstart":"0","idlist":["40453629","40224132","40224084","40182463","40182449","39902424","39631964","39071
✅ Success! Found 20465 results


In [75]:
# === TODO (you code this) ===
# Goal: Paginate through ESearch results to collect all PMIDs.
# Hints:
# 1) Loop with incrementing retstart until no more IDs returned
# 2) Use batch_size from config for retmax
# 3) Check total count to know when to stop
# Acceptance:
# - Function get_all_pmids(query, batch_size) -> list[str]
# - Returns all PMIDs, not just first batch
# - No duplicates

def get_all_pmids(query, batch_size=500):
    """Paginate ESearch to collect all PMIDs for a query."""
    pmids = []
    retstart = 0
    
    # First call to get total count
    first_result = esearch(query, retmax=batch_size, retstart=0)
    total_count = int(first_result['esearchresult']['count'])
    
    # NCBI hard limit: can only retrieve first 10,000 results
    MAX_RESULTS = 10000
    if total_count > MAX_RESULTS:
        print(f"⚠️  Found {total_count:,} results, but NCBI limits to {MAX_RESULTS:,}")
        print(f"    Will retrieve first {MAX_RESULTS:,} results")
        total_count = MAX_RESULTS
    else:
        print(f"📊 Found {total_count:,} total results")
    
    # Add first batch
    pmids.extend(first_result['esearchresult']['idlist'])
    retstart += batch_size
    
    # Progress bar
    from tqdm import tqdm
    pbar = tqdm(total=total_count, initial=batch_size, desc="Fetching PMIDs")
    
    # Continue pagination (with safety limit)
    while retstart < total_count and retstart < MAX_RESULTS:
        result = esearch(query, retmax=batch_size, retstart=retstart)
        id_list = result['esearchresult']['idlist']
        
        if not id_list:
            break
            
        pmids.extend(id_list)
        retstart += batch_size
        pbar.update(len(id_list))
    
    pbar.close()
    
    print(f"✅ Collected {len(pmids):,} unique PMIDs")
    return pmids

# Test with 2024 (smaller dataset)
test_query = build_query(2024, config['term_template'])
pmids_2024 = get_all_pmids(test_query, batch_size=500)

# Verify
print(f"\nFirst 5 PMIDs: {pmids_2024[:5]}")
print(f"Last 5 PMIDs: {pmids_2024[-5:]}")
print(f"Any duplicates? {len(pmids_2024) != len(set(pmids_2024))}")


🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...
✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"23246","retmax":"500","retstart":"0","idlist":["41194911","41134706","41064704","41059869","41050229","41017877","40984692","4097
⚠️  Found 23,246 results, but NCBI limits to 10,000
    Will retrieve first 10,000 results


Fetching PMIDs:   5%|▌         | 500/10000 [00:00<?, ?it/s]

🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


Fetching PMIDs:  10%|█         | 1000/10000 [00:00<00:14, 600.68it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"23246","retmax":"500","retstart":"500","idlist":["39925995","39925693","39925692","39925688","39922652","39919934","39917717","39
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


...

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"23246","retmax":"500","retstart":"8500","idlist":["39147631","39147374","39147018","39146990","39146707","39146660","39146659","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


Fetching PMIDs:  95%|█████████▌| 9500/10000 [00:16<00:00, 535.99it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"23246","retmax":"500","retstart":"9000","idlist":["39104346","39104345","39104344","39104343","39104341","39104340","39104338","3
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


Fetching PMIDs: 100%|█████████▉| 9999/10000 [00:17<00:00, 541.49it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"23246","retmax":"499","retstart":"9500","idlist":["39064157","39064100","39064069","39064052","39063952","39063878","39063725","3
✅ Collected 9,999 unique PMIDs

First 5 PMIDs: ['41194911', '41134706', '41064704', '41059869', '41050229']
Last 5 PMIDs: ['39029891', '39029614', '39029336', '39029289', '39029217']
Any duplicates? False


## EFetch: Download XML

Use `efetch.fcgi` to retrieve MEDLINE XML for batches of PMIDs.


In [76]:
# === TODO (you code this) ===
# Goal: Fetch MEDLINE XML for a list of PMIDs.
# Hints:
# 1) Join PMIDs with commas for 'id' parameter
# 2) Request 'xml' retmode (not json)
# 3) Return raw XML text, not JSON
# Acceptance:
# - Function efetch(pmids:list) -> str
# - Returns XML string starting with <?xml

def efetch(pmids):
    """Fetch MEDLINE XML for a batch of PMIDs."""
    params = {
        'db': 'pubmed',
        'id': ','.join(pmids),
        'retmode': 'xml',
        'rettype': 'medline'
    }
    response = eutils_get('efetch.fcgi', params)
    return response.text


## Main Ingestion Loop

For each year:
1. Build query
2. Get PMIDs
3. Fetch XML in batches
4. Save to `data/raw/pubmed_{year}_{batch}.xml`


In [77]:
# === TODO (you code this) ===
# Goal: Orchestrate full ingestion for all years.
# Hints:
# 1) Loop through year range from config
# 2) For each year: build query, get all PMIDs, chunk and fetch XML
# 3) Save files as pubmed_{year}_{chunk:04d}.xml
# 4) Skip existing files for resumability
# Acceptance:
# - Function ingest_years(config) processes all years
# - Files written to ../data/raw/
# - Re-running skips existing files
# - Progress printed for each year
import time

def ingest_years(config):
    """Ingest all configured years into data/raw/."""
    raw_dir = Path('../data/raw')
    raw_dir.mkdir(parents=True, exist_ok=True)
    
    start_year = config['start_year']
    end_year = config['end_year']
    batch_fetch = config['batch_fetch']
    batch_ids = config['batch_ids']  # ← ADD THIS
    
    for year in range(start_year, end_year + 1):
        print(f"\n=== Processing year {year} ===")
        
        # Build query and get all PMIDs
        query = build_query(year, config['term_template'])
        pmids = get_all_pmids(query, batch_ids)  # ← FIX: Use get_all_pmids!
        
        if not pmids:
            print(f"No PMIDs found for {year}")
            continue
        
        print(f"Found {len(pmids)} PMIDs for {year}")
        
        # Chunk PMIDs and fetch XML in batches
        for i in range(0, len(pmids), batch_fetch):
            chunk_num = i // batch_fetch
            output_file = raw_dir / f"pubmed_{year}_{chunk_num:04d}.xml"
            
            # Skip existing files for resumability
            if output_file.exists():
                print(f"  Skipping existing file: {output_file.name}")
                continue
            
            # Get batch of PMIDs
            batch = pmids[i:i + batch_fetch]
            print(f"  Fetching batch {chunk_num} ({len(batch)} PMIDs)...")
            
            # Fetch XML and save
            xml_data = efetch(batch)
            output_file.write_text(xml_data, encoding='utf-8')
            print(f"  ✅ Saved {output_file.name}")

# Run ingestion (uncomment when ready)
ingest_years(config)



=== Processing year 2018 ===
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...
✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"17808","retmax":"500","retstart":"0","idlist":["29369573","35187394","33312071","33206890","32520276","32520270","32520263","3252
⚠️  Found 17,808 results, but NCBI limits to 10,000
    Will retrieve first 10,000 results


Fetching PMIDs:   5%|▌         | 500/10000 [00:00<?, ?it/s]

🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


Fetching PMIDs:  10%|█         | 1000/10000 [00:00<00:16, 557.92it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"17808","retmax":"500","retstart":"500","idlist":["30672992","30672990","30672989","30672988","30672987","30672986","30672985","30
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


...

Fetching PMIDs:  95%|█████████▌| 9500/10000 [00:16<00:00, 536.64it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"19731","retmax":"500","retstart":"9000","idlist":["40422626","40422625","40422623","40422622","40422621","40422619","40422618","4
🔍 Calling: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi
📝 Query preview: (dentistry[MeSH Terms] OR "dental care"[MeSH Terms] OR "oral health"[MeSH Terms] OR "dental implants...


Fetching PMIDs: 100%|█████████▉| 9999/10000 [00:17<00:00, 542.21it/s]

✅ Status code: 200
📦 Content type: application/json; charset=UTF-8
📄 First 200 chars of response:
{"header":{"type":"esearch","version":"0.3"},"esearchresult":{"count":"19731","retmax":"499","retstart":"9500","idlist":["40381477","40381475","40381268","40381068","40381050","40380999","40380958","4
✅ Collected 9,999 unique PMIDs
Found 9999 PMIDs for 2025
  Fetching batch 0 (200 PMIDs)...


  ✅ Saved pubmed_2025_0000.xml
  Fetching batch 1 (200 PMIDs)...
  ✅ Saved pubmed_2025_0001.xml
  Fetching batch 2 (200 PMIDs)...
  ✅ Saved pubmed_2025_0002.xml
  Fetching batch 3 (200 PMIDs)...
  ✅ Saved pubmed_2025_0003.xml
  Fetching batch 4 (200 PMIDs)...
  ✅ Saved pubmed_2025_0004.xml
  Fetching batch 5 (200 PMIDs)...
  ✅ Saved pubmed_2025_0005.xml
  Fetching batch 6 (200 PMIDs)...
  ✅ Saved pubmed_2025_0006.xml
  Fetching batch 7 (200 PMIDs)...
  ✅ Saved pubmed_2025_0007.xml
  Fetching batch 8 (200 PMIDs)...
  ✅ Saved pubmed_2025_0008.xml
  Fetching batch 9 (200 PMIDs)...
  ✅ Saved pubmed_2025_0009.xml
  Fetching batch 10 (200 PMIDs)...
  ✅ Saved pubmed_2025_0010.xml
  Fetching batch 11 (200 PMIDs)...
  ✅ Saved pubmed_2025_0011.xml
  Fetching batch 12 (200 PMIDs)...
  ✅ Saved pubmed_2025_0012.xml
  Fetching batch 13 (200 PMIDs)...
  ✅ Saved pubmed_2025_0013.xml
  Fetching batch 14 (200 PMIDs)...
  ✅ Saved pubmed_2025_0014.xml
  Fetching batch 15 (200 PMIDs)...
  ✅ Saved pubmed_20

## QA Checklist

Before moving to the next notebook, verify:

- [ ] Counts per year are non-zero
- [ ] Files written to `data/raw/` directory
- [ ] XML files are valid (spot-check by opening one)
- [ ] Re-running the cell skips existing files (resumability)
- [ ] No rate-limit errors from NCBI

### Sanity Check: Count Files


In [78]:
# === TODO (you code this) ===
# Goal: Count and verify downloaded XML files.
# Hints:
# 1) Use Path.glob() to find all .xml files
# 2) Parse filenames to extract year
# 3) Show total count and breakdown by year
# Acceptance:
# - Prints total file count
# - Shows count per year (use Counter)

# TODO: count files in ../data/raw/*.xml
from collections import Counter

raw_dir = Path('../data/raw')

xml_files = list(raw_dir.glob("*.xml"))
print(f"Total XML files: {len(xml_files)}")

# Extract year from filename (format: pubmed_YYYY_NNNN.xml)
year_counts = Counter()
for file in xml_files:
    parts = file.stem.split('_')
    if len(parts) >= 2:
        year = parts[1]
        year_counts[year] += 1

print("\nBreakdown by year:")
for year in sorted(year_counts.keys()):
    print(f"  {year}: {year_counts[year]} files")


Total XML files: 400

Breakdown by year:
  2018: 50 files
  2019: 50 files
  2020: 50 files
  2021: 50 files
  2022: 50 files
  2023: 50 files
  2024: 50 files
  2025: 50 files


## 🧘 Reflection Log

**What did you learn in this session?**
- Built a reliable PubMed ingestion pipeline by chaining modular helpers (`load_config`, `build_query`, `esearch`, `get_all_pmids`, `efetch`, `ingest_years`).
- Saw how configurable term templates let us widen or narrow the dental scope without rewriting code.
- Understood how MEDLINE XML is stored for provenance so later notebooks can parse titles, abstracts, publication types, and MeSH terms.

**What challenges did you encounter?**
- Hit NCBI maintenance windows and learned to retry or wait gracefully.
- Fought JSON decoding errors caused by multiline YAML queries until we normalized whitespace.
- Discovered PubMed’s 10K result cap and updated pagination to respect it.

**How will this improve Periospot AI?**
- Guarantees reproducible metadata ingestion (~400 XML files across 2018–2025) ready for downstream labeling.
- Provides a dependable way to refresh the dataset whenever new dentistry literature appears.
- Sets the foundation for robust evidence triage models that can trust their upstream data.
